### Final Project PanCancer Atlas
***The Cancer Genome Atlas Pan-Cancer Atlas was a program initiated to categorize 33 cancer types from over 10,000 samples.  Much work has been done in this collaborative effort using a variety of molecular markers - mutations, aneuploidy, copy number, gene expression, methylations, proteomics etc.  In a flagship article on the Cell-of-Origin for the Pan-Cancer Atlas, researchers identified homogenous cancer specific groups based on characteristics associated with cell-of-origin and tissue-specific markers as well as cancer types that had considerable overlap across organ systems. The ability to classify tumors by tissue of origin and to target expression unique to tissue-specific tumors has the potential to enhance site-specific targeting of tumors if novel markers can be identified.  While it is widely accepted that there are "oncogenic" pathways that repeatedly appear across cancer types, there is potentially a diverse landscape of pathways that may be driving or sustaining tumor growth within a given tissue.  Using RNA sequencing data collected and batch corrected from the large sample size and breadth of cancers in the Pan-Cancer Atlas, can we identify organ-specific markers that lend credence to the plausibility that tissues do in fact hold clues to site-specifc treatment options?***

### We will need to install and import several libraries.
***Using pandas for dataframe manipulation, matplotlib for visualization, numpy for mathematical manipulation and array operations, sklearn and pytorch for ML modeling.  Additionally, we will need to use the library rpy2 which is a python wrapper of the edgeR package used in gene expression normalization***



In [ ]:
# Install required packages (once per virtual environment)
%pip install -q numpy pandas matplotlib seaborn scikit-learn tensorflow[and-cuda] xgboost rpy2
%reset -f

In [24]:
import pandas as pd
import numpy as np 
import matplotlib as plt
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
import random
import time

##### Load the datasets
***We will load the gene expression data as well as the paired clinical data.***

In [2]:
# Load the Pan-Cancer gene expression data from the tsv file
PANCAN_df = pd.read_csv('EBPlusPlusAdjustPANCAN_IlluminaHiSeq_RNASeqV2.geneExp.tsv', sep='\t')


### Inspect the gene expression data and generate some helper functions.

***We want to see what the raw data looks like so we can perform the appropriate munging steps.***

##### The PanCancer Atlas dataset requires several preprocessing steps to perform meaningful analysis.  
***There are several tumor sample types denoted in the sample ID.  For example, the primary site of the tumor is denoted in the string index 14-15.  Additionally, the string vector for gene IDs contains both hardware ID and Gene names.  Finally, when dealing with a dataset that is composed of expression level data, it is necessary to adjust the counts for the depth of the sequencing in a sample relative to all other samples, followed by log transformation, and gene-wise scaling to Z-scores.***

In [16]:
# Describe the gene expression data.

def inspection(exprs):
    """We want to get information from the data to assess how to approach munging.
    Args: pdDataFrame

    """
    if not isinstance(exprs, pd.DataFrame):
        raise TypeError("Input must be a pandas DataFrame.")

    print("Dimensions of the data:", exprs.shape)
    print()

    try:
        print("Data types of each column:")
        print(exprs.dtypes)
        print()

        print("Number of missing values in each column:")
        print(exprs.isnull().sum())
        print()

        print("Summary statistics for numerical columns:")
        print(exprs.describe())
        print()
        
        print("Unique values in categorical columns:")
        for column in exprs.select_dtypes(include=['object', 'category']):
            print(f"{column}: {exprs[column].nunique()} unique values")
        print()

        print("Random sample of the dataframe:")
        print(exprs.sample(5))
        print()

        print("Memory usage of the dataframe:")
        print(exprs.memory_usage(deep=True))
        print()

        print("Sample of the dataframe:")
        print(exprs.head())
    except Exception as e:
        print(f"Error during dataframe inspection: {e}")


# Describe the landscape of the data for the normalization step.

def analyze_value_distribution(df):
    """
    Analyze the distribution of non-zero, null, zero, and negative values in a DataFrame,
    focusing only on numeric columns.

    Args: pdDataFrame 

    """
    if not isinstance(df, pd.DataFrame):
        raise TypeError("Input must be a pandas DataFrame.")

    # Select only numeric columns for analysis
    numeric_df = df.select_dtypes(include=[np.number])

    # Count the non-zero values
    non_zero_count = (numeric_df != 0).sum().sum()
    
    # Count the NA or null values in the numeric columns
    null_count = numeric_df.isnull().sum().sum()
    
    # Count the zero values
    zero_count = (numeric_df == 0).sum().sum()
    
    # Count the negative values
    negative_count = (numeric_df < 0).sum().sum()

    # Calculate and print the percentages
    if non_zero_count > 0:  # Prevent division by zero
        null_percentage = (null_count / non_zero_count) * 100
        zero_percentage = (zero_count / non_zero_count) * 100
        negative_percentage = (negative_count / non_zero_count) * 100

        print(f"Percentage of null or NA relative to the total non-zero counts: {null_percentage:.2f}%")
        print(f"Percentage of zeros relative to the total non-zero counts: {zero_percentage:.2f}%")
        print(f"Percentage of negatives relative to the total non-zero counts: {negative_percentage:.2f}%")
    else:
        print("No non-zero counts to calculate percentages.")


# Convert the null and negative values to 0
        
def convert_null_and_negative_to_zero(df):
    """
    Convert null or negative values in numeric columns to zero.

    Args: pdDataFrame

    Returns: pd.DataFrame: The modified DataFrame with the null and negative values set to zero.

    """
    if not isinstance(df, pd.DataFrame):
        raise TypeError("Input must be a pandas DataFrame.")

    # Define a function to convert negative and null values to zero
    def convert_to_zero(x):
        if pd.isnull(x) or x < 0:
            return 0
        return x

    # Apply the function to the numeric columns of the DataFrame
    numeric_cols = df.select_dtypes(include=[np.number])
    df[numeric_cols.columns] = numeric_cols.applymap(convert_to_zero)

    return df

# Clean the Numeric Columns - Samples - to select the first 15 chars, selecting only primary samples '-01', and deduplicating those columns.

def truncate_and_deduplicate_column_names(df):
    """
    Truncate all numeric column names to the first 15 characters, then select and deduplicate columns ending with '-01'.

    Args:
        df (pd.DataFrame): The DataFrame to process.

    Returns:
        pd.DataFrame: The modified DataFrame with truncated and deduplicated numeric column names ending with '-01'.
    """
    if not isinstance(df, pd.DataFrame):
        raise TypeError("Input must be a pandas DataFrame.")

    # Ensure the gene_id column is retained
    gene_id_col = df['gene_id'] if 'gene_id' in df.columns else None

    # Truncate all numeric column names to the first 15 characters, excluding gene_id
    numeric_cols = df.select_dtypes(include=[np.number]).columns.difference(['gene_id'])
    truncated_names = {col: col[:15] for col in numeric_cols}
    df = df.rename(columns=truncated_names)

    # Select truncated numeric columns ending with '-01', excluding gene_id
    cols_to_process = [col for col in df.columns if col.endswith('-01') and col != 'gene_id']
    df = df[cols_to_process + ['gene_id']]

    # Deduplicate column names efficiently, excluding gene_id
    duplicate_counts = df.columns.value_counts()
    duplicated_columns = duplicate_counts[duplicate_counts > 1].index.difference(['gene_id'])

    for col in duplicated_columns:
        cols_to_keep = [c for c in df.columns if c == col and c != 'gene_id']
        if len(cols_to_keep) > 1:
            chosen_col = random.choice(cols_to_keep)
            cols_to_drop = set(cols_to_keep) - {chosen_col}
            df = df.drop(columns=cols_to_drop)

    # Ensure gene_id column is still in the DataFrame
    if gene_id_col is not None:
        df['gene_id'] = gene_id_col

    return df


def extract_gene_id(df):
    """
    Extract the 'gene_id' column from the DataFrame and return it along with the modified DataFrame.

    Args: pd.DataFrame

    Returns: pd.DataFrame
        tuple: A tuple containing the extracted 'gene_id' column and the modified DataFrame.
    """
    if 'gene_id' in df.columns:
        gene_id_col = df['gene_id']
        df = df.drop('gene_id', axis=1)
        return gene_id_col, df
    else:
        raise KeyError("The 'gene_id' column does not exist in the DataFrame.")

def reinsert_gene_id(df, gene_id_col):
    """
    Reinsert the 'gene_id' column into the DataFrame at the first position.

    Args:
        df (pd.DataFrame): The DataFrame to process.
        gene_id_col (pd.Series): The 'gene_id' column to reinsert.

    Returns:
        pd.DataFrame: The modified DataFrame with 'gene_id' as the first column.
    """
    df.insert(0, 'gene_id', gene_id_col)
    return df

##### What measures need to be taken to homogenize the coverage?

Based on the inspection of the dataset, summary statistics of the numerical columns indicate null values for certain samples as well as negative values.  For the sequencing count normalization across samples, we can set the null values and negative values to zero.  

In [ ]:
inspection(PANCAN_df)

***To run the normalization step adjusting for depth-of-sequencing we need to do some accounting of the zero and null values.  For RNAseq, there are thresholds for acceptable levels of conversion of null and negative counts to zero.  We will proceed with this conversion for the sake of a comprehensive analysis.***

In [ ]:
analyze_value_distribution(PANCAN_df)

In [4]:
convert_null_and_negative_to_zero(PANCAN_df)

/tmp/ipykernel_13675/3971558193.py:109: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[numeric_cols.columns] = numeric_cols.applymap(convert_to_zero)


,gene_id,TCGA-OR-A5J1-01A-11R-A29S-07,TCGA-OR-A5J2-01A-11R-A29S-07,TCGA-OR-A5J3-01A-11R-A29S-07,TCGA-OR-A5J5-01A-11R-A29S-07,TCGA-OR-A5J6-01A-31R-A29S-07,TCGA-OR-A5J7-01A-11R-A29S-07,TCGA-OR-A5J8-01A-11R-A29S-07,TCGA-OR-A5J9-01A-11R-A29S-07,TCGA-OR-A5JA-01A-11R-A29S-07,...,TCGA-CG-4449-01A-01R-1157-13,TCGA-CG-4462-01A-01R-1157-13,TCGA-CG-4465-01A-01R-1157-13,TCGA-CG-4466-01A-01R-1157-13,TCGA-CG-4469-01A-01R-1157-13,TCGA-CG-4472-01A-01R-1157-13,TCGA-CG-4474-01A-02R-1157-13,TCGA-CG-4475-01A-01R-1157-13,TCGA-CG-4476-01A-01R-1157-13,TCGA-CG-4477-01A-01R-1157-13
0,?|100130426,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,?|100133144,3.2661,2.6815,1.7301,0.0000,0.0000,1.1673,1.4422,0.0000,4.4556,...,4.358154,5.676995,5.219350,14.846708,20.115492,6.997533,18.311906,12.057112,18.628740,17.874417
2,?|100134869,3.9385,8.9948,6.5650,1.5492,4.4709,6.0529,2.2876,1.3599,5.0581,...,2.656360,3.342794,2.423442,5.055287,11.626054,13.654193,7.417109,11.585177,11.482418,14.919338
3,?|10357,149.1350,81.0777,86.4879,53.9117,66.9063,103.5060,94.9316,78.1955,69.2389,...,633.299781,294.018042,686.569179,563.573453,1039.307597,639.238135,742.479964,506.336449,712.452165,703.713324
4,?|10431,2034.1000,1304.9300,1054.6600,2350.8900,1257.9900,1866.4300,995.0270,1762.1200,1213.5300,...,1202.538277,644.002317,1181.884532,663.885074,647.530395,1297.152549,1152.909807,1375.495774,971.893874,1736.988111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20526,ZYG11A|440590,0.4803,31.4052,0.5925,11.6189,7.8240,85.4392,0.4144,2.3799,1.0571,...,20.923873,1.839530,2.916935,239.014921,1.845753,3.268489,17.164493,3.756246,0.301440,217.431795
20527,ZYG11B|79699,648.4150,1166.0200,806.3990,553.8340,795.8120,520.6580,556.1540,913.1870,805.4970,...,1322.386301,1025.213701,814.306556,907.845035,953.276441,905.046317,757.811259,927.963540,845.677334,859.078048
20528,ZYX|7791,1841.0200,3059.9900,2655.6100,2367.9300,708.0710,855.1940,10924.6000,2122.1600,1939.2200,...,2783.898049,4960.431833,3447.701267,978.304677,2789.057736,3359.241568,4264.469081,3103.609391,3302.569055,2497.814797
20529,ZZEF1|23140,1157.5400,1895.9900,1482.4500,1140.2000,796.3710,897.7140,1095.7300,1003.6200,904.8630,...,1284.992478,2054.896390,2420.047163,1302.821382,1119.313995,1740.926312,2702.668453,1370.141309,1915.477072,1247.130940


##### **Let us rerun the inspection function to assess whether the conversions were the appropriate remedy**

In [5]:
inspection(PANCAN_df)

Dimensions of the data: (20531, 11070)

Data types of each column:
gene_id                          object
TCGA-OR-A5J1-01A-11R-A29S-07    float64
TCGA-OR-A5J2-01A-11R-A29S-07    float64
TCGA-OR-A5J3-01A-11R-A29S-07    float64
TCGA-OR-A5J5-01A-11R-A29S-07    float64
                                 ...   
TCGA-CG-4472-01A-01R-1157-13    float64
TCGA-CG-4474-01A-02R-1157-13    float64
TCGA-CG-4475-01A-01R-1157-13    float64
TCGA-CG-4476-01A-01R-1157-13    float64
TCGA-CG-4477-01A-01R-1157-13    float64
Length: 11070, dtype: object

Number of missing values in each column:
gene_id                         0
TCGA-OR-A5J1-01A-11R-A29S-07    0
TCGA-OR-A5J2-01A-11R-A29S-07    0
TCGA-OR-A5J3-01A-11R-A29S-07    0
TCGA-OR-A5J5-01A-11R-A29S-07    0
                               ..
TCGA-CG-4472-01A-01R-1157-13    0
TCGA-CG-4474-01A-02R-1157-13    0
TCGA-CG-4475-01A-01R-1157-13    0
TCGA-CG-4476-01A-01R-1157-13    0
TCGA-CG-4477-01A-01R-1157-13    0
Length: 11070, dtype: int64

Summary statistics 

***Run the selection and deduplication step.***

In [14]:
# Start the timer
start_time = time.time()

# Call the function
modified_df = truncate_and_deduplicate_column_names(PANCAN_df)

# Stop the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time
print(f"The function took {elapsed_time} seconds to run.")


The function took 4.155496835708618 seconds to run.


In [15]:
inspection(modified_df)

Dimensions of the data: (20531, 9715)

Data types of each column:
TCGA-OR-A5J1-01    float64
TCGA-OR-A5J2-01    float64
TCGA-OR-A5J3-01    float64
TCGA-OR-A5J5-01    float64
TCGA-OR-A5J6-01    float64
                    ...   
TCGA-CG-4474-01    float64
TCGA-CG-4475-01    float64
TCGA-CG-4476-01    float64
TCGA-CG-4477-01    float64
gene_id             object
Length: 9715, dtype: object

Number of missing values in each column:
TCGA-OR-A5J1-01    0
TCGA-OR-A5J2-01    0
TCGA-OR-A5J3-01    0
TCGA-OR-A5J5-01    0
TCGA-OR-A5J6-01    0
                  ..
TCGA-CG-4474-01    0
TCGA-CG-4475-01    0
TCGA-CG-4476-01    0
TCGA-CG-4477-01    0
gene_id            0
Length: 9715, dtype: int64

Summary statistics for numerical columns:
       TCGA-OR-A5J1-01  TCGA-OR-A5J2-01  TCGA-OR-A5J3-01  TCGA-OR-A5J5-01  \
count     20531.000000     20531.000000     20531.000000     20531.000000   
mean        855.442271       929.652398      1054.063489      1007.788021   
std        4177.520062      4752.50

***The data appears to be ready for the normalization step!  The input to the edgeR package can only contain numeric columns. So, lets save the gene_ids to a list, remove it from the dataframe, run the normalization step, and reinsert after norm step is completed.***

In [ ]:
gene_ids, modified_df = extract_gene_id(modified_df)


##### Run Gene Expression Normalization with Python wrapper of edgeR
***In order to run this step we needed to clean the gene expression matrix of null and negative values.  Recall:  we are adjusting counts across samples based on a geometric mean normalization factor.***

In [19]:
# Start the timer
start_time = time.time()

# Activate the pandas conversion automatically
pandas2ri.activate()



# Import the edgeR package
edgeR = importr('edgeR')

# Convert the pandas DataFrame to an R data frame
r_data = pandas2ri.py2rpy(modified_df)

# Create a DGEList object (assuming counts are in r_data)
dge = edgeR.DGEList(counts=r_data)

# Perform TMM normalization
dge = edgeR.calcNormFactors(dge)

# You can now proceed with the normalized data in R, or convert it back to Python
normalized_counts = edgeR.cpm(dge, log=True)

# Stop the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time
print(f"The function took {elapsed_time} seconds to run.")

The function took 411.57851099967957 seconds to run.


***Let's inspect the data once more. First, convert back to a pandas dataframe.***

In [22]:
norm_df = pd.DataFrame(normalized_counts)
inspection(norm_df)

Dimensions of the data: (20531, 9702)

Data types of each column:
0       float64
1       float64
2       float64
3       float64
4       float64
         ...   
9697    float64
9698    float64
9699    float64
9700    float64
9701    float64
Length: 9702, dtype: object

Number of missing values in each column:
0       0
1       0
2       0
3       0
4       0
       ..
9697    0
9698    0
9699    0
9700    0
9701    0
Length: 9702, dtype: int64

Summary statistics for numerical columns:
               0             1             2             3             4     \
count  20531.000000  20531.000000  20531.000000  20531.000000  20531.000000   
mean       2.110365      2.204193      2.186538      2.291430      2.094356   
std        3.895462      3.845894      3.850500      3.719026      3.940397   
min       -3.332306     -3.332306     -3.332306     -3.332306     -3.332306   
25%       -2.103420     -1.891587     -1.825397     -1.349403     -2.381889   
50%        2.879239      3.056743 

***There appears to be a reduction in samples after the Normalization step.  We went from 9714 to 9702 samples.  Not sure where those 12 samples disappeared to!***

In [23]:
# set the gene_ids as the indices for the rows of the dataframe.
norm_df.index = gene_ids

##### Now we need to transpose the normalized counts so that the genes are columns and samples are rows, and perform a Z-score scaling of each gene across samples.

In [25]:
# transpose the dataframe
Tnorm_df = norm_df.transpose()

scaler = StandardScaler()
Znorm_df = scaler.fit_transform(Tnorm_df)

Znorm_df = pd.DataFrame(Znorm_df, columns=Tnorm_df.columns, index=Tnorm_df.index)

In [26]:
inspection(Znorm_df)

Dimensions of the data: (9702, 20531)

Data types of each column:
gene_id
?|100130426      float64
?|100133144      float64
?|100134869      float64
?|10357          float64
?|10431          float64
                  ...   
ZYG11A|440590    float64
ZYG11B|79699     float64
ZYX|7791         float64
ZZEF1|23140      float64
ZZZ3|26009       float64
Length: 20531, dtype: object

Number of missing values in each column:
gene_id
?|100130426      0
?|100133144      0
?|100134869      0
?|10357          0
?|10431          0
                ..
ZYG11A|440590    0
ZYG11B|79699     0
ZYX|7791         0
ZZEF1|23140      0
ZZZ3|26009       0
Length: 20531, dtype: int64

Summary statistics for numerical columns:
gene_id   ?|100130426   ?|100133144   ?|100134869       ?|10357       ?|10431  \
count    9.702000e+03  9.702000e+03  9.702000e+03  9.702000e+03  9.702000e+03   
mean    -6.915419e-12 -3.058366e-15 -4.687151e-17 -1.462391e-14  2.603712e-14   
std      1.000052e+00  1.000052e+00  1.000052e+00

## From here we use the Sample IDs to join with the Clinical or Phenotypic data or any other data from the PanCancer Atlas.  
***Though I ran out of time, I will perform some machine learning dimensionality reduction, and classification on a more complex array.  The gene expression data is officially clean and ready for downstream analysis.  One good option to reduce feature space on the gene expression is to perform gene-set enrichment analysis to produce single values representing biological processes.***